# Machine Learning Applications for Health (COMP90089_2022_SM2)
# Tutorial 3: Unsupervised Learning using MIMIC-IV data.

* We are going to play with two types of Clustering Algorithms: K-Means and DB-Scan
* The Python lybrary will be sklearn. To check their documentation, please click [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and [here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN). 

The K-Means was based on [this](https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters/notebook)  tutorial. Feel free to read in entirely.

The DBSCAN was based on [this](https://medium.com/@tarammullin/dbscan-2788cfce9389) tutorial. 


Set up the **environment**:

In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

#plotly imports
#!pip install plotly==5.10.0
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
pio.renderers.default = "colab"

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

import warnings
warnings.filterwarnings('ignore')

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [2]:
# authenticate
auth.authenticate_user()

In [5]:
# Set up environment variables
project_id = 'CHANGE'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'


## Data 


We'll use a cohort derived from MIMIC-IV.

* Use the query bellow to search the data in the **BigQuery Platform**.
* We are retrieving patients with **Sepsis**: A life-threatening complication caused by the body's response to an infection. When your immune system goes into **overdrive in response to an infection**, sepsis may develop as a result

In [ ]:
##We are retrieving patients using sepsis3 Table and joining it to patients Table.

df = run_query("""
SELECT sep.subject_id,sep.sofa_score,sep.respiration,sep.coagulation,sep.liver,sep.cardiovascular,sep.cns,sep.renal,pt.dod
FROM `physionet-data.mimiciv_derived.sepsis3` as sep
INNER JOIN `physionet-data.mimiciv_hosp.patients` as pt
ON sep.subject_id = pt.subject_id
ORDER BY subject_id
""")
print(df)

In [ ]:
##Retain the information columns for K-Means and DBSCAN

pd.set_option('display.max_columns', None) ##This is only to show all column when printing a DataFrame

patients = df['subject_id'] #Patients information will be separated
dod = df['dod'] #Patients Date of Death information will be separated too.

#Final Data set to work with:
dataset = df.drop(['subject_id','dod'], axis = 1)

## Check for missing values
print(dataset.isnull().sum())

data_km = dataset.copy()
data_db = dataset.copy()

## K-Means 

Main **Parameters** of the K-Means Function:

**n_clusters** = the number of desired clusters. 

**init** = is the method for initialisation. (the default is ‘k-means++’, can also be ‘random’)

**n_init** = how many times to run the k-means clustering algorithms independently with different random centroids to choose the final model (minimising the intra-cluster Sum of Squared Errors(SSE)).

**max_iter** =  the maximum number of iterations for each single run.

### How to chose the number of clusters? 

* We can do that cheching the intra-distance parameter (the smaller, the better).
The atribute **_inertia** can measure it. 

In [ ]:
intradistance = [] 
for number_of_clusters in range(1, 11): 
    kmeans = KMeans(n_clusters = number_of_clusters, random_state = 0)
    kmeans.fit(data_km) 
    intradistance.append(kmeans.inertia_)
print(intradistance)

## Elbow plot for decision:
ks = [1,2,3,4,5,6,7,8,9,10]
print(ks)
plt.plot(ks, intradistance)

### Using the Elbow plot to identify the optimal number of clusters. Can we can choose = 2 ?

In [ ]:
#Let's try:

number_of_clusters = 2

kmeans = KMeans(n_clusters = number_of_clusters, random_state = 0)
kmeans.fit(data_km) 
    
#Find which cluster each data-point belongs to
clusters = kmeans.predict(data_km)

#Add the cluster information as a new column to our DataFrame
data_km["Cluster"] = clusters

print(data_km.groupby(['Cluster']).sum())

### Visualisation using PCA

PCA is an algorithm that is used for **dimensionality reduction** - meaning, informally, that it can take in a DataFrame with many columns and return a DataFrame with a reduced number of columns that still retains much of the information from the columns of the original DataFrame. **The columns of the DataFrame produced from the PCA procedure are called Principal Components**.

In [36]:
#plotX is copy of the current DataFrame to be plotted:
plotX = data_km

#PCA with 2 principal components for a 2-D visualisation:
pca_2d = PCA(n_components=2)

#And this DataFrame contains 2 principal components that will aid us
#in visualizing our clusters in 
PCs_2d = pd.DataFrame(pca_2d.fit_transform(plotX.drop(["Cluster"], axis=1)))
PCs_2d.columns = ["PC1_2d", "PC2_2d"]


#We concatenate these newly created DataFrames to plotX so that they can be used by plotX as columns.
plotX = pd.concat([plotX,PCs_2d], axis=1, join='inner')

#Now we divide plotX into 2 new DataFrames.
#Each of these new DataFrames will hold all of the values contained in exacltly one of the clusters.

cluster0 = plotX[plotX["Cluster"] == 0]
cluster1 = plotX[plotX["Cluster"] == 1]


## Calculate the mean value (Centroids) for each cluster to be seen in the Plot Figure:

centr = [[cluster0["PC1_2d"].mean(), cluster0["PC2_2d"].mean()],
         [cluster1["PC1_2d"].mean(), cluster1["PC2_2d"].mean()]]

centr_x = [x for x,y in centr]
centr_y = [y for x,y in centr]

In [ ]:
#This is needed so we can display plotly plots properly
# init_notebook_mode(connected=True) #If you are running it locally as a jupyter notebook
enable_plotly_in_cell() #If you are running it in Google Colab as a jupyter notebook

#trace1 is for 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_2d"],
                    y = cluster0["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_2d"],
                    y = cluster1["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)


# Include the centroids
centroids = go.Scatter(x = centr_x,
                       y = centr_y,
                      mode = "markers",
                      name = 'centroids',
                      marker = dict(color = 'black'),
                      text = None)

data = [trace1, trace2, centroids]

title = "Visualizing Clusters in 2 Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

# Plot the data:
fig = dict(data = data, layout = layout)
iplot(fig)


* We can use **BoxPlot to visually look for differences** between columns (For each Cluster):

In [ ]:

boxplot = data_km.boxplot(column=['respiration','liver','cardiovascular'], by='Cluster')





## DBScan


Main **Parameters** of the DBSCAN Function:

eps = is the maximum distance between two samples for one to be considered as in the neighborhood of the other.
(default is 0.5)

min_samples = The number of samples (or total weight) in a neighborhood for a point to be considered as a core point.
(default is 5)

metric = The metric to use when calculating distance between instances in a feature array.
(default is 'euclidean')
    

**Go ahead and play with the epsilon values and min sample values below to check the effects they have on the clusters**

In [ ]:
#Recap the data:

print(data_db.head())


Calculate the average distance between each point in the data set and its 14 nearest neighbors (my selected MinPts value = 2* num columns).

In [39]:
neighbors = NearestNeighbors(n_neighbors=14)
neighbors_fit = neighbors.fit(data_db)
distances, indices = neighbors_fit.kneighbors(data_db)

Sort distance values by ascending value and plot: it will produce a k-distance elbow plot 

In [ ]:
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [41]:
# Compute DBSCAN
db = DBSCAN(eps=1.25, min_samples=14) ## <- CHANGE THESE VALUES AND PLOT THE EFFECTS 

clustering_labels = db.fit_predict(data_db)
data_db['labels'] = clustering_labels

### Evaluate Model Performance — Mean Silhouette Coefficient

The Silhouette Coefficient is bounded between 1 and -1. The best value is 1, the worst is -1. A higher score indicates that the model has better defined, more dense clusters. Values close to 0 indicate overlapping clusters, while negative values usually indicate that data points have been assigned to the wrong clusters.

In [ ]:
from sklearn import metrics

metrics.silhouette_score(data_db, data_db['labels'])


In [ ]:
labels = db.labels_
print(set(labels))
print(len(set(labels)))